## Deliverable 2. Create a Customer Travel Destinations Map.

In [56]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [57]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint Anthony,US,45.0205,-93.2180,1.40,50,40,16.11,scattered clouds
1,1,Ilhabela,BR,-23.7781,-45.3581,77.47,64,39,4.94,scattered clouds
2,2,Cape Town,ZA,-33.9258,18.4232,72.00,78,40,11.50,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.85,73,100,11.72,light rain
4,4,Nikolskoye,RU,59.7035,30.7861,10.40,72,75,15.66,snow


In [58]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [59]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ilhabela,BR,-23.7781,-45.3581,77.47,64,39,4.94,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,77.85,73,100,11.72,light rain
9,9,Nioro,GM,13.3500,-15.7500,77.97,23,100,8.97,overcast clouds
12,12,Bubaque,GW,11.2833,-15.8333,82.13,57,45,12.88,scattered clouds
15,15,Tautira,PF,-17.7333,-149.1500,78.80,73,75,14.97,light rain
19,19,Vaini,TO,-21.2000,-175.2000,77.00,83,75,5.75,broken clouds
21,21,Butaritari,KI,3.0707,172.7902,81.07,80,100,20.92,light rain
24,24,Arauca,CO,7.0847,-70.7591,88.38,51,100,3.94,overcast clouds
27,27,Atuona,PF,-9.8000,-139.0333,80.13,78,100,23.31,overcast clouds
39,39,Palmas Bellas,PA,9.2319,-80.0867,89.01,63,100,10.78,overcast clouds


In [60]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                185
City                   185
Country                182
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
dtype: int64

In [61]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
Preferred_cities_df =preferred_cities_df.dropna()
#Preferred_cities_df.columns

In [62]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = Preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ilhabela,BR,77.47,scattered clouds,-23.7781,-45.3581,
3,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,
9,Nioro,GM,77.97,overcast clouds,13.3500,-15.7500,
12,Bubaque,GW,82.13,scattered clouds,11.2833,-15.8333,
15,Tautira,PF,78.80,light rain,-17.7333,-149.1500,
19,Vaini,TO,77.00,broken clouds,-21.2000,-175.2000,
21,Butaritari,KI,81.07,light rain,3.0707,172.7902,
24,Arauca,CO,88.38,overcast clouds,7.0847,-70.7591,
27,Atuona,PF,80.13,overcast clouds,-9.8000,-139.0333,
39,Palmas Bellas,PA,89.01,overcast clouds,9.2319,-80.0867,


In [63]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [64]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"]= hotel_df["Hotel Name"].replace ("",np.nan)

clean_hotel_df= hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ilhabela,BR,77.47,scattered clouds,-23.7781,-45.3581,Ilha Flat Hotel
3,Rikitea,PF,77.85,light rain,-23.1203,-134.9692,Pension Maro'i
12,Bubaque,GW,82.13,scattered clouds,11.2833,-15.8333,"Casa Dora, Bubaque"
15,Tautira,PF,78.80,light rain,-17.7333,-149.1500,Pension Oaoa
19,Vaini,TO,77.00,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
...,...,...,...,...,...,...,...
657,Bossangoa,CF,76.82,clear sky,6.4926,17.4552,LA PERLE DE BOSTON
659,Ponnani,IN,77.00,haze,10.7670,75.9252,Rouba Residency
660,Karratha,AU,77.88,scattered clouds,-20.7377,116.8463,ibis Styles Karratha
664,Sao Benedito Do Rio Preto,BR,85.41,broken clouds,-3.3336,-43.5283,João de Barro Hotel


In [69]:
# 8a. Create the output File (CSV)

output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [70]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [68]:
# 11a. Add a marker layer for each city to the map. 
#max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
#             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig
# 11b. Display the figure


Figure(layout=FigureLayout(height='420px'))